In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

data = fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])

In [2]:
img_pipeline = Pipeline([("mm_scaler", MinMaxScaler())])
y = dfData["target"]
dfData = dfData.drop("target",axis=1)
X = dfData.copy()
X_transf = img_pipeline.fit_transform(X)

In [3]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in stratSplit.split(X_transf, y):
    X_train = X_transf[train_index]
    X_test = X_transf[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

In [4]:
log_reg = LogisticRegression(C=1e5, max_iter=100)
log_reg.fit(X_train, y_train)
print("Test set performance:",log_reg.score(X_test, y_test))

Test set performance: 0.9244285714285714


/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [5]:
#Once you have your final model it can be useful to to re-train it on the full dataset
log_reg.fit(X_transf, y)
log_reg.score(X_transf, y)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9347

## Saving your model

In [6]:
# Option 1 joblib
from sklearn.externals import joblib
joblibModelName = 'ultimateMNISTClassifierJoblibSave.pkl'
joblib.dump(log_reg, joblibModelName)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['ultimateMNISTClassifierJoblibSave.pkl']

In [7]:
# Option 2 Pickle
import pickle
pickleModelName = 'ultimateMNISTClassifierPickleSave.pkl'
with open(pickleModelName, "wb") as out:
    pickle.dump(log_reg, out)

In [8]:
# If you did grid search remember you can access the best estimator by usinng .best_estimator_

## Saving your pipeline

In [9]:
from sklearn.externals import joblib
pipelineName = 'ultimateMNISTClassifierPipeline.pkl'
joblib.dump(img_pipeline, pipelineName)

['ultimateMNISTClassifierPipeline.pkl']

## Loading Pipeline

In [10]:
pipelineName = 'ultimateMNISTClassifierPipeline.pkl'
loaded_pipeline = joblib.load(pipelineName)
X_trans_loaded = loaded_pipeline.transform(X)

## Loading Model

In [11]:
# Option 1 joblib
modelName = 'ultimateMNISTClassifierJoblibSave.pkl'
loaded_log_clf = joblib.load(modelName)
loaded_log_clf.score(X_trans_loaded, y)

0.9347

In [12]:
# Option 2 pickle
modelName = 'ultimateMNISTClassifierPickleSave.pkl'
with open(modelName, 'rb') as inModel:
    loaded_log_clf = pickle.load(inModel)
loaded_log_clf.score(X_trans_loaded, y)

0.9347